In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sbn
sbn.set()
import matplotlib.pyplot as plt
import re

from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

# Document Analysis - Loading the Data

In [3]:
fetch_20newsgroups?

In [4]:
newsgroups = fetch_20newsgroups()   #Get the training subset

In [5]:
newsgroups.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [6]:
print(newsgroups.target_names[newsgroups.target[1]], "\n\n", newsgroups.data[1])

comp.sys.mac.hardware 

 From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [7]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as sw

In [8]:
print(sw)
print(len(sw))

frozenset({'see', 'anyone', 'him', 'yourself', 'how', 'any', 'nine', 'across', 'around', 'ourselves', 'twelve', 'eight', 'never', 'noone', 'cry', 'for', 'either', 'by', 'ie', 'one', 'third', 'after', 'here', 'she', 'anyway', 'hereby', 're', 'up', 'whose', 'yours', 'not', 'due', 'whereafter', 'nevertheless', 'eleven', 'hundred', 'my', 'afterwards', 'some', 'seem', 'though', 'call', 'everyone', 'has', 'himself', 'other', 'put', 'itself', 'something', 'last', 'do', 'three', 'couldnt', 'least', 'empty', 'often', 'therefore', 'such', 'you', 'fill', 'serious', 'seeming', 'except', 'front', 'there', 'this', 'find', 'former', 'inc', 'may', 'part', 'via', 'were', 'but', 'thin', 'found', 'wherein', 'many', 'into', 'done', 'must', 'indeed', 'first', 'well', 'yet', 'however', 'whole', 'among', 'several', 'thereby', 'whether', 'same', 'toward', 'whence', 'would', 'so', 'her', 'although', 'hers', 'out', 'until', 'off', 'now', 'sometime', 'can', 'nor', 'amount', 'he', 'are', 'nothing', 'it', 'onto', 

# Classification

In [35]:
def generateWordDict(emailStr):
    emailStr = re.sub(r"[.:(),@><-`]", " ", emailStr)
    emailStr = re.sub(r"[0-9]", " ", emailStr)
    emailStr = emailStr.replace("?", " ").replace("$", " ").replace("/", " ").replace("\\", " ").replace("'", " ").replace("\"", " ").replace("*", " ").replace("=", " ")
    emailStr = emailStr.replace("(", " ").replace(")", " ").replace("_", " ").replace("~", " ").replace("&", " ").replace("#", " ").replace("[", " ").replace("]", " ")
    wordList = emailStr.lower().split()
    
    finalWordList = []
    for word in wordList:
        if word not in sw:
            finalWordList.append(word)
            
    wordDict = {}
    for word in finalWordList:
        if word in wordDict.keys():
            wordDict[word] += 1
        else:
            wordDict[word] = 1
            
    return wordDict


# print(generateWordDict(newsgroups.data[1]))

In [36]:
email_bag_o_words = []

for email in newsgroups.data:
    email_bag_o_words.append(generateWordDict(email))

In [17]:
print(email_bag_o_words[2])

{'folks': 1, 'sooner': 1, 'lies': 1, 'mb': 1, 'dirt': 1, 'corner': 1, 'engineering': 2, 'hopefully': 1, 'ec': 1, 'people': 1, 'anybody': 3, 'way': 1, ';': 1, 'questions': 2, 'finally': 1, 't': 3, 'realize': 1, 'gave': 1, 'c': 1, 'll': 1, 'distribution': 1, 'enemies': 1, 'helpful': 1, 'ghost': 1, 'convictions': 1, 'email': 1, 'opinions': 2, 'w': 1, 'hellcats': 1, 'uses': 1, 'feel': 1, 'question': 1, 'heard': 3, 'd': 1, 'duo': 1, 'ones': 1, 'played': 1, 'real': 1, 'daily': 1, 'pb': 1, 'somebody': 2, 'bit': 1, 'summer': 1, 'usa': 1, 'probably': 1, 'wondering': 1, 'figured': 1, 'thomas': 1, 'time': 1, 'worth': 1, 'looking': 1, 'drops': 1, 'impression': 1, 'round': 1, 'yea': 1, 'computer': 2, 'taking': 1, 'life': 1, 'good': 1, 'k': 1, 'like': 1, 'disk': 2, 'solicit': 1, 'machine': 2, 'use': 1, 'recently': 1, 'line': 1, 'day': 2, 'nietzsche': 1, 'swing': 1, 'access': 1, 'sooo': 1, 'introductions': 1, 'subjective': 1, 'lines': 1, 'answer': 1, 'appearence': 1, 'dangerous': 1, 'subject': 1, 'pl

In [12]:
# print(np.sign(np.dot(email_bag_o_words[0], email_bag_o_words[2])))

In [13]:
# X, y = newsgroups.data, newsgroups.target
# Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

In [37]:
email_bag_o_words

[{'-': 1,
  '----': 2,
  '-door': 1,
  'addition': 1,
  'ark': 1,
  'aryland': 1,
  'body': 1,
  'brought': 1,
  'bumper': 1,
  'called': 1,
  'car': 5,
  'day': 1,
  'doors': 1,
  'e-mail': 1,
  'early': 1,
  'edu': 2,
  'engine': 1,
  'enlighten': 1,
  'erxst': 1,
  'f': 1,
  'funky': 1,
  'hanks': 1,
  'history': 1,
  'ines': 1,
  'info': 1,
  'know': 1,
  'late': 1,
  'lerxst': 1,
  'looked': 1,
  'looking': 1,
  'model': 1,
  'n': 1,
  'neighborhood': 1,
  'niversity': 1,
  'ntp-': 1,
  'ollege': 1,
  'ost': 1,
  'osting-': 1,
  'production': 1,
  'rac': 1,
  'really': 1,
  'rest': 1,
  'rganization': 1,
  'ricklin': 1,
  'rom': 1,
  's': 3,
  'saw': 1,
  'separate': 1,
  'small': 1,
  'specs': 1,
  'sports': 1,
  't': 2,
  'tellme': 1,
  'thing': 1,
  'this!': 1,
  'ubject': 1,
  'umd': 2,
  'wam': 2,
  'wondering': 1,
  'years': 1},
 {'-': 2,
  'acceleration': 1,
  'adapters': 1,
  'add': 2,
  'answered': 1,
  'ashington': 1,
  'attained': 1,
  'base': 1,
  'brave': 1,
  'brief'